# Modelos de regresión con datos numéricos, imágenes y mixto

Montamos la unidad de GDrive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [13]:
# Copiamos los ficheros de GDrive al entorno de ejecución
!cp /content/drive/My\ Drive/x_train_num.npy x_train_num.npy
!cp /content/drive/My\ Drive/x_train_img.npy x_train_img.npy
!cp /content/drive/My\ Drive/x_val_num.npy x_val_num.npy
!cp /content/drive/My\ Drive/x_val_img.npy x_val_img.npy
!cp /content/drive/My\ Drive/x_test_num.npy x_test_num.npy
!cp /content/drive/My\ Drive/x_test_img.npy x_test_img.npy
!cp /content/drive/My\ Drive/y_train_reg.npy y_train_reg.npy
!cp /content/drive/My\ Drive/y_val_reg.npy y_val_reg.npy
!cp /content/drive/My\ Drive/y_test_reg.npy y_test_reg.npy

In [14]:
# Cargamos los ficheros a variables de nuestro entorno
import numpy as np
x_train_num = np.load('x_train_num.npy')
x_train_img = np.load('x_train_img.npy')
x_val_num = np.load('x_val_num.npy')
x_val_img = np.load('x_val_img.npy')
x_test_num = np.load('x_test_num.npy')
x_test_img = np.load('x_test_img.npy')
y_train_reg = np.load('y_train_reg.npy')
y_val_reg = np.load('y_val_reg.npy')
y_test_reg = np.load('y_test_reg.npy')

In [15]:
# Mostramos la forma de los arrays para ver que no haya errores
print("datos numericos de train:", x_train_num.shape)
print("datos de imágenes de train:",x_train_img.shape)
print("valores de train:",y_train_reg.shape)
print("datos numericos de validación:",x_val_num.shape)
print("datos de imágenes de validación:",x_val_img.shape)
print("valores de validación:",y_val_reg.shape)
print("datos numericos de test:",x_test_num.shape)
print("datos de imágenes de test:",x_test_img.shape)
print("valores de test:",y_test_reg.shape)

datos numericos de train: (7125, 30)
datos de imágenes de train: (7125, 64, 64, 3)
valores de train: (7125,)
datos numericos de validación: (814, 30)
datos de imágenes de validación: (814, 64, 64, 3)
valores de validación: (814,)
datos numericos de test: (2687, 30)
datos de imágenes de test: (2687, 64, 64, 3)
valores de test: (2687,)


In [16]:
# Hacemos una transformación logarítmica a los precios
y_train_reg = np.log10(y_train_reg)
y_val_reg = np.log10(y_val_reg)
y_test_reg = np.log10(y_test_reg)

### Modelo de regresión sólo con datos numéricos

Creamos un modelo secuencial con una primera capa densa con 8 neuronas. En esa primera capa hay que pasar la dimensión y la función de activación es relu. La siguiente capa que añadimos ya es más pequeña con 4 neuronas y también función de activación relu. La última capa al ser una red para regresión tiene sólo una neurona y función de activación linear. Es exactamente igual al de clasificación pero cambiando esta última capa.

In [17]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(8, input_dim=x_train_num.shape[1], activation="relu"))
model.add(Dense(4, activation="relu"))
model.add(Dense(1, activation="linear"))

Vamos a usar el optimizador Adam y la función de pérdidas el error cuadrático medio.

In [18]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_squared_error", optimizer=opt)
# train the model
print("[INFO] training model...")
model.fit(x=x_train_num, y=y_train_reg, 
	validation_data=(x_val_num, y_val_reg),
	epochs=50, batch_size=8)

[INFO] training model...
Epoch 1/50
891/891 [==============================] - 3s 3ms/step - loss: 34.5914 - val_loss: 0.5594
Epoch 2/50
891/891 [==============================] - 3s 3ms/step - loss: 0.5371 - val_loss: 0.2552
Epoch 3/50
891/891 [==============================] - 3s 3ms/step - loss: 0.1635 - val_loss: 0.1211
Epoch 4/50
891/891 [==============================] - 3s 3ms/step - loss: 0.0888 - val_loss: 0.0898
Epoch 5/50
891/891 [==============================] - 3s 3ms/step - loss: 0.0767 - val_loss: 0.0871
Epoch 6/50
891/891 [==============================] - 3s 3ms/step - loss: 0.0760 - val_loss: 0.0870
Epoch 7/50
891/891 [==============================] - 3s 3ms/step - loss: 0.0760 - val_loss: 0.0870
Epoch 8/50
891/891 [==============================] - 3s 3ms/step - loss: 0.0760 - val_loss: 0.0869
Epoch 9/50
891/891 [==============================] - 3s 3ms/step - loss: 0.0760 - val_loss: 0.0870
Epoch 10/50
891/891 [==============================] - 3s 3ms/step - loss:

Comprobamos el modelo con los datos de test.

In [19]:
scores = model.evaluate(x_test_num, y_test_reg, verbose=0)
print('Loss: %.3f' % scores)

Loss: 0.078


Vemos como en la época 6 habíamos llegago a un mínimo en la función de pérdidas y ya no se producen mejoras.

### Modelo de regresión sólo con imágenes

En esta ocación usaremos una red neuronal convolucional para poder trabajar con las imágenes. Crearemos el modelo mediante un método funcional en vez de secuencial como anteriormente. Dependiendo de los filtros que pasemos creará diferentes capas, en este caso crearemos una de 16, otra de 32 y otra de 64. En la primera iteración indicamos la dimensión de los datos al igual que hacíamos en el modelo secuencial. Es igual que el modelo de clasificación pero cambiando la última capa.

In [20]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input
# Tamaño de las imágenes
inputShape = (64, 64, 3)
chanDim = -1
# Filtros que vamos a aplicar en las capas convolucionales
filters=(16, 32, 64)
inputs = Input(shape=inputShape)
# Recorremos el bucle creando las diferentes capas
for (i, f) in enumerate(filters):
		# si es la primera capa indicamos el inputshape
  if i == 0:
	  x = inputs
	# CONV => BN => RELU => POOL
  x = Conv2D(f, (3, 3), padding="same")(x)
  x = BatchNormalization(axis=chanDim)(x)
  x = Activation("relu")(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
	
# Aplanamos los datos y ponemos una capa densa de 16 neuronas con activación relu
x = Flatten()(x)
x = Dense(16)(x)
x = BatchNormalization(axis=chanDim)(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
# Añadimos otra capa densa de 4 neuronas y activación relu
x = Dense(4)(x)
x = Activation("relu")(x)
# Añadimos la última capa con 1 neuronas y función linear para la regresión 
x = Dense(1, activation="linear")(x)
# Construimos la red neuronal
model = Model(inputs, x)

Vamos a usar el optimizador Adam y la función de pérdidas el error cuadrático medio.

In [21]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_squared_error", optimizer=opt)
# entrenamos el modelo
print("[INFO] training model...")
model.fit(x=x_train_img, y=y_train_reg, 
    validation_data=(x_val_img, y_val_reg),
    epochs=50, batch_size=8)

[INFO] training model...
Epoch 1/50
891/891 [==============================] - 7s 7ms/step - loss: 0.4865 - val_loss: 0.0835
Epoch 2/50
891/891 [==============================] - 6s 7ms/step - loss: 0.1042 - val_loss: 0.0818
Epoch 3/50
891/891 [==============================] - 6s 7ms/step - loss: 0.0816 - val_loss: 0.0804
Epoch 4/50
891/891 [==============================] - 6s 7ms/step - loss: 0.0760 - val_loss: 0.0793
Epoch 5/50
891/891 [==============================] - 6s 7ms/step - loss: 0.0732 - val_loss: 0.0780
Epoch 6/50
891/891 [==============================] - 6s 7ms/step - loss: 0.0710 - val_loss: 0.0766
Epoch 7/50
891/891 [==============================] - 6s 7ms/step - loss: 0.0702 - val_loss: 0.0826
Epoch 8/50
891/891 [==============================] - 6s 7ms/step - loss: 0.0689 - val_loss: 0.0749
Epoch 9/50
891/891 [==============================] - 6s 7ms/step - loss: 0.0668 - val_loss: 0.0763
Epoch 10/50
891/891 [==============================] - 6s 7ms/step - loss: 

Comprobamos el modelo con los datos de test.

In [22]:
scores = model.evaluate(x_test_img, y_test_reg, verbose=0)
print('Loss: %.3f' % scores)

Loss: 0.076


En esta ocasión vemos como el modelo tiene overfitting, se ha aprendido los datos de entrenamiento y se ajusta muy bien a estos, pero no a los de test o validación. Podemos comprobarlo por la diferencia entre cada una de las funciones de pérdidas.

### Modelo de regresión sólo con datos mixtos, imágenes y numéricos.

En esta ocasión usaremos los dos tipos de datos que hemos visto anteriormente. Vamos a crear un modelo con dos ramas:


1.   Rama 1: una red neuronal para analizar los datos numéricos, exactamente igual a la que hemos usado antes, pero sin la capa final de regresión.
2.   Rama 2: una red neuronal convolucional para trabajar con las imágenes, exactamente igual a la anterior, pero sin la capa final de regresión.

Posteriormente combinaremos ambas ramas creando otra capa densa de 4 neuronas con función de activación relu y la capa final de regresión con 1 neurona y función de activación linear.


In [23]:
# Creamos la rama numérica
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

mlp = Sequential()
mlp.add(Dense(8, input_dim=x_train_num.shape[1], activation="relu"))
mlp.add(Dense(4, activation="relu"))

In [24]:
# Creamos la rama que trabaja con imágenes
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input
# Tamaño de las imágenes
inputShape = (64, 64, 3)
chanDim = -1
# Filtros que vamos a aplicar en las capas convolucionales
filters=(16, 32, 64)
inputs = Input(shape=inputShape)
# Recorremos el bucle creando las diferentes capas
for (i, f) in enumerate(filters):
		# si es la primera capa indicamos el inputshape
  if i == 0:
	  x = inputs
	# CONV => BN => RELU => POOL
  x = Conv2D(f, (3, 3), padding="same")(x)
  x = BatchNormalization(axis=chanDim)(x)
  x = Activation("relu")(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
	
# Aplanamos los datos y ponemos una capa densa de 16 neuronas con activación relu
x = Flatten()(x)
x = Dense(16)(x)
x = BatchNormalization(axis=chanDim)(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
# Añadimos otra capa densa de 4 neuronas y activación relu
x = Dense(4)(x)
x = Activation("relu")(x)
# Construimos la red neuronal
cnn = Model(inputs, x)

In [25]:
from tensorflow.keras.layers import concatenate
# Combinamos ambas redes
combinedInput = concatenate([mlp.output, cnn.output])

In [26]:
# Añadimos la capa densa con 4 neuronas y la de 3 para la clasificación
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

In [27]:
# Construimos el modelo que combina datos numéricos e imágenes
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

In [28]:
# Usamos de nuevo el optimizados Adam y la función de pérdidas error cuadrático medio y compilamos el modelo
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_squared_error", optimizer=opt)

In [29]:
# Entrenamos el modelo
print("[INFO] training model...")
model.fit(
	x=[x_train_num, x_train_img], y=y_train_reg,
	validation_data=([x_val_num, x_val_img], y_val_reg),
	epochs=50, batch_size=8)


[INFO] training model...
Epoch 1/50
891/891 [==============================] - 7s 8ms/step - loss: 84.1229 - val_loss: 0.1462
Epoch 2/50
891/891 [==============================] - 7s 8ms/step - loss: 0.1200 - val_loss: 0.0940
Epoch 3/50
891/891 [==============================] - 7s 8ms/step - loss: 0.0821 - val_loss: 0.1287
Epoch 4/50
891/891 [==============================] - 7s 8ms/step - loss: 0.0767 - val_loss: 0.0881
Epoch 5/50
891/891 [==============================] - 7s 7ms/step - loss: 0.0727 - val_loss: 0.0909
Epoch 6/50
891/891 [==============================] - 7s 8ms/step - loss: 0.0684 - val_loss: 0.0846
Epoch 7/50
891/891 [==============================] - 7s 8ms/step - loss: 0.0672 - val_loss: 0.0648
Epoch 8/50
891/891 [==============================] - 7s 8ms/step - loss: 0.0623 - val_loss: 0.0611
Epoch 9/50
891/891 [==============================] - 7s 8ms/step - loss: 0.0571 - val_loss: 0.0573
Epoch 10/50
891/891 [==============================] - 7s 8ms/step - loss:

Comprobamos el modelo con los datos de test.

In [30]:
scores = model.evaluate([x_test_num, x_test_img], y_test_reg, verbose=0)
print('Loss: %.3f' % scores)

Loss: 0.071


Creo que ha ocurrido lo mismo que antes y tenemos overfitting, ya que se puede obervar como la función de pérdidas va reduciendo en train época a época, sin embargo en validación permanece casi constante.